In [1]:
import matplotlib as mpl

import matplotlib.pyplot as plt
#import seaborn as sns
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

import networkx as nx

import pymysql
import pickle
import json

import numpy as np
from sklearn.decomposition import PCA

import time
import hdbscan
import umap

import spacy
from gensim.models import Word2Vec

import random
import re
from pmids2vec import pmids2vec, pmids2vec_titlesOnly
from pmids2corpus import pmids2corpus    #  todo integrate this into pmids2vec

### control parameters

In [2]:
path2clusteredPMIDs = 'data_processing_feb2020/pmids_2018_200k.json'
model_export_prefix = '2018_200k_win2'
embed_titles_text = False
embed_abstracts_text = True

### get metadata

In [3]:
with open(path2clusteredPMIDs, 'r') as f:
    data = json.load(f)

N_samples = len(data['pmids'])

In [4]:
### embed abstract text

In [5]:
# todo force samples to have the same size, test impact of vocabulary ? 
#    or maybe just test variance over multiple samplesets to check for effects of sample size asymmetry

# load pmids

if embed_abstracts_text:

    for sample_id in range(N_samples):
        model_names = []

        with open(path2clusteredPMIDs, 'r') as f:
            data = json.load(f)

        #print(data['pmids'])   # i_sample, i_cluster
        sample_pmids = []

        clustered_pmids = data['pmids'][str(sample_id)]
        for cluster_id_str, pmids_list in clustered_pmids.items():

            model_export_path = 'data_processing_feb2020/abstracts_{}_cluster{}_{}.model'.format(
                                    sample_id, cluster_id_str, model_export_prefix)
            model_names.append(model_export_path)

            print('calling pmids2vec...')
            pmids2vec(pmids_list, model_export_path)
            
    print('--------------')
    

calling pmids2vec...
SQL join executed in 254.49029088020325 s
SQL results fetched and cast in 0.0714254379272461 s
training word2vec model...
params: 20 dimensions, 2 window size, 165 min count
elapsed: 54.688549518585205
calling pmids2vec...
SQL join executed in 174.67576503753662 s
SQL results fetched and cast in 0.05671882629394531 s
training word2vec model...
params: 20 dimensions, 2 window size, 79 min count
elapsed: 17.6289541721344
calling pmids2vec...
SQL join executed in 120.07616376876831 s
SQL results fetched and cast in 0.025022029876708984 s
training word2vec model...
params: 20 dimensions, 2 window size, 45 min count
elapsed: 8.174963235855103
calling pmids2vec...
SQL join executed in 140.47887206077576 s
SQL results fetched and cast in 0.01224207878112793 s
training word2vec model...
params: 20 dimensions, 2 window size, 23 min count
elapsed: 4.830634117126465
calling pmids2vec...
SQL join executed in 163.42041087150574 s
SQL results fetched and cast in 0.02452564239501

In [6]:
# recreate & save the corpus for good measure

if embed_abstracts_text:

    for sample_id in range(N_samples):
        model_names = []

        with open(path2clusteredPMIDs, 'r') as f:
            data = json.load(f)
        sample_pmids = []

        clustered_pmids = data['pmids'][str(sample_id)]
        for cluster_id_str, pmids_list in clustered_pmids.items():

            corpus_export_path = 'data_processing_feb2020/abstracts_{}_cluster{}_{}_corpus.json'.format(
                                    sample_id, cluster_id_str, model_export_prefix)
            model_names.append(corpus_export_path)

            print('calling pmids2corpus...')
            pmids2corpus(pmids_list, corpus_export_path)
            
    print('---------------')


calling pmids2corpus...
SQL join executed in 151.8175024986267 s
SQL results fetched and cast in 0.05471158027648926 s
saving new work to data_processing_feb2020/abstracts_0_cluster0_2018_200k_win2_corpus.json
calling pmids2corpus...
SQL join executed in 138.4910340309143 s
SQL results fetched and cast in 0.0637669563293457 s
saving new work to data_processing_feb2020/abstracts_0_cluster1_2018_200k_win2_corpus.json
calling pmids2corpus...
SQL join executed in 131.97399854660034 s
SQL results fetched and cast in 0.01126408576965332 s
saving new work to data_processing_feb2020/abstracts_0_cluster2_2018_200k_win2_corpus.json
calling pmids2corpus...
SQL join executed in 127.63900756835938 s
SQL results fetched and cast in 0.010601282119750977 s
saving new work to data_processing_feb2020/abstracts_0_cluster3_2018_200k_win2_corpus.json
calling pmids2corpus...
SQL join executed in 130.2751533985138 s
SQL results fetched and cast in 0.009467363357543945 s
saving new work to data_processing_feb

In [7]:
### embed titles only

In [8]:
# titles only (saves the corpus while training the titles-only models)

# todo force samples to have the same size, test impact of vocabulary ? 
#    or maybe just test variance over multiple samplesets to check for effects of sample size asymmetry

# load pmids

if embed_titles_text:
    
    for sample_id in range(N_samples):
        model_names = []

                                 #  todo need much larger set of pmids since this is titles only
        with open(path2clusteredPMIDs, 'r') as f:
            data = json.load(f)

        #print(data['pmids'])   # i_sample, i_cluster
        sample_pmids = []
        clustered_pmids = data['pmids'][str(sample_id)]
        for cluster_id_str, pmids_list in clustered_pmids.items():

            model_export_path = 'data_processing_feb2020/titles_{}_cluster{}_{}'.format(
                    sample_id, cluster_id_str, model_export_prefix)

            print('calling pmids2vec...')
            pmids2vec_titlesOnly(pmids_list, model_export_path)
            
        print('---------------------------------')
